In [1]:
#import os
#os.environ["OMP_NUM_THREADS"] = "14"
import veloxchem as vlx
from pandas import *
from rdkit import Chem

In [2]:
data = read_csv("nucleophilicity_with_names.csv")
smiles_list = data['smiles'].tolist()

In [3]:
mol = vlx.Molecule.read_smiles(smiles_list[1])
mol.show()
rd_mol = Chem.MolFromSmiles(smiles_list[1])
charge = Chem.GetFormalCharge(rd_mol)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
%%time
results = []
for i in range(len(smiles_list)):
    mol = vlx.Molecule.read_smiles(smiles_list[i])
    rd_mol = Chem.MolFromSmiles(smiles_list[i])
    charge = Chem.GetFormalCharge(rd_mol)
    mol.set_charge(charge)
    basis = vlx.MolecularBasis.read(mol, 'def2-svp')
    scf_drv = vlx.ScfRestrictedDriver()
    scf_drv.ri_coulomb = True
    scf_drv.grid_level = 2
    scf_drv.level_shifting = 0.0
    scf_drv.xcfun = "pbe"
    scf_drv.dispersion = True
    scf_drv.conv_thresh = 1e-4
    scf_drv.ostream.mute()
    result = scf_drv.compute(mol, basis)
    if result == None:
        scf_drv.level_shifting = 1.0
        result = scf_drv.compute(mol, basis)
        if result == None:
            print("SCF failed")
        else:
            results.append(result['scf_energy'])
            print(str("Energy: " + str(result['scf_energy'])) + " of structure " + str(i+1) + " with level shifting 1.0")
    else:
        results.append(result['scf_energy'])
        print(str("Energy: " + str(result['scf_energy'])) + " of structure " + str(i+1))
print("Finished")

Energy: -468.5810536471125 of structure 1
Energy: -614.7549224748448 of structure 2
Energy: -365.72631336746247 of structure 3
Energy: -388.31057166965854 of structure 4
Energy: -349.6955785858482 of structure 5
Energy: -271.2197867214962 of structure 6
Energy: -251.37048130058585 of structure 7
Energy: -403.50558814506775 of structure 8
Energy: -330.3930974548011 of structure 9
Energy: -322.44434910970705 of structure 10
Energy: -542.2532670096715 of structure 11
Energy: -311.64556330844863 of structure 12
Energy: -365.7379244162782 of structure 13
Energy: -557.8469150567362 of structure 14
Energy: -479.37303598688777 of structure 15
Energy: -390.10835546503404 of structure 16
Energy: -479.3801331258543 of structure 17
Energy: -537.7161124159855 of structure 18
Energy: -385.5737142302164 of structure 19
Energy: -483.4204042235712 of structure 20
Energy: -517.8817190770114 of structure 21
Energy: -365.484391764214 of structure 22
Energy: -597.073282176791 of structure 23
Energy: -448.7

In [ ]:
%%time
opt_geometries = []
opt_energies = []
for i in range(10):
    mol = vlx.Molecule.read_smiles(smiles_list[i])
    rd_mol = Chem.MolFromSmiles(smiles_list[i])
    charge = Chem.GetFormalCharge(rd_mol)
    mol.set_charge(charge)
    basis = vlx.MolecularBasis.read(mol, 'def2-svp')
    scf_drv = vlx.ScfRestrictedDriver()
    scf_drv.ri_coulomb = True
    scf_drv.grid_level = 2
    scf_drv.level_shifting = 1.0
    scf_drv.xcfun = "pbe"
    scf_drv.ostream.mute()
    scf_drv.dispersion = True
    scf_drv.conv_thresh = 1e-4
    result = scf_drv.compute(mol, basis)

    if result == None:
        scf_drv.level_shifting = 1.0
        result = scf_drv.compute(mol, basis)
        if result == None:
            print("SCF failed")
            continue
        
    opt_drv = vlx.OptimizationDriver(scf_drv)
    scf_drv.level_shifting = 1.0
    opt_drv.conv_energy = 1e-04
    opt_drv.conv_drms = 1e-02
    opt_drv.conv_dmax = 2e-02
    opt_drv.conv_grms = 4e-03
    opt_drv.conv_gmax = 8e-03
    opt_results = opt_drv.compute(mol, basis, result)
    opt_geometries.append(opt_results['final_geometry'])
    opt_energies.append(opt_results['opt_energies'][-1])
    print(str("Energy: " + str(opt_results['opt_energies'][-1])) + " of optimized structure " + str(i+1) + ". Optimized in " + str(len(opt_results['opt_energies'])) + " iterations.")
print("Finished")

Energy: -468.58246975792656 of optimized structure 1. Optimized in 3 iterations.
Energy: -614.7617750532547 of optimized structure 2. Optimized in 12 iterations.


In [8]:
results = []
mol = vlx.Molecule.read_smiles(smiles_list[1194])
rd_mol = Chem.MolFromSmiles(smiles_list[1194])
charge = Chem.GetFormalCharge(rd_mol)
mol.show()
print(charge)
mol.set_charge(charge)
basis = vlx.MolecularBasis.read(mol, 'def2-svp')
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ri_coulomb = True
scf_drv.grid_level = 2
scf_drv.xcfun = "pbe"
scf_drv.dispersion = True
scf_drv.conv_thresh = 1e-4
result = scf_drv.compute(mol, basis)
if result == None:
    scf_drv.level_shifting = 1.0
    result = scf_drv.compute(mol, basis)
    results.append(result['scf_energy'])
    print(str("Energy: " + str(result['scf_energy'])) + " of structure " + str(42+1))
else:
    results.append(result['scf_energy'])
    print(str("Energy: " + str(result['scf_energy'])) + " of structure " + str(42+1))

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

-1
***********                                                                                                               
* Warning * DFT grid level is below the recommended value. Please double check.                                           
***********                                                                                                               
                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Kohn-Sham                                            
                   Initial Guess Model             : Superposition of Atomic Densities                                    
             

In [4]:
%%time
results = []
for i in range(10):
    mol = vlx.Molecule.read_smiles(smiles_list[i])
    rd_mol = Chem.MolFromSmiles(smiles_list[i])
    charge = Chem.GetFormalCharge(rd_mol)
    mol.set_charge(charge)
    basis = vlx.MolecularBasis.read(mol, 'def2-svp')
    scf_drv = vlx.ScfRestrictedDriver()
    scf_drv.ri_coulomb = True
    scf_drv.grid_level = 2
    scf_drv.xcfun = "pbe"
    scf_drv.dispersion = True
    scf_drv.conv_thresh = 1e-4
    scf_drv.ostream.mute()
    result = scf_drv.compute(mol, basis)
    results.append(result['scf_energy'])
    print(str("Energy: " + str(result['scf_energy'])) + " of structure " + str(i+1))
print("Finished")

Energy: -468.58064676183756 of structure 1
Energy: -614.7546038101879 of structure 2
Energy: -365.7260033499709 of structure 3
Energy: -388.31033980601063 of structure 4
Energy: -349.6953538878379 of structure 5
Energy: -271.21961809157153 of structure 6
Energy: -251.37028577309502 of structure 7
Energy: -403.50544022495126 of structure 8
Energy: -330.39277374551847 of structure 9
Energy: -322.4440825951374 of structure 10
Finished
CPU times: user 1min 13s, sys: 23.4 s, total: 1min 37s
Wall time: 17.2 s


In [10]:
%%time
results = []
for i in range(10):
    mol = vlx.Molecule.read_smiles(smiles_list[i])
    rd_mol = Chem.MolFromSmiles(smiles_list[i])
    charge = Chem.GetFormalCharge(rd_mol)
    mol.set_charge(charge)
    basis = vlx.MolecularBasis.read(mol, 'def2-svp')
    scf_drv = vlx.ScfRestrictedDriver()
    scf_drv.ri_coulomb = False
    scf_drv.grid_level = 2
    scf_drv.xcfun = "pbe"
    scf_drv.dispersion = True
    scf_drv.conv_thresh = 1e-4
    scf_drv.ostream.mute()
    result = scf_drv.compute(mol, basis)
    results.append(result['scf_energy'])
    print(str("Energy: " + str(result['scf_energy'])) + " of structure " + str(i+1))
print("Finished")

Energy: -468.5803288012151 of structure 1
Energy: -614.7544700799415 of structure 2
Energy: -365.72607613013463 of structure 3
Energy: -388.31017211608923 of structure 4
Energy: -349.6951489310269 of structure 5
Energy: -271.2193460512624 of structure 6
Energy: -251.37023265380816 of structure 7
Energy: -403.5053283790967 of structure 8
Energy: -330.3797021572891 of structure 9
Energy: -322.4438233113809 of structure 10
Finished
CPU times: user 11min 35s, sys: 1min 30s, total: 13min 6s
Wall time: 1min 12s


In [5]:
%%time
results = []
for i in range(10):
    mol = vlx.Molecule.read_smiles(smiles_list[i])
    rd_mol = Chem.MolFromSmiles(smiles_list[i])
    charge = Chem.GetFormalCharge(rd_mol)
    mol.set_charge(charge)
    basis = vlx.MolecularBasis.read(mol, 'def2-svpd')
    scf_drv = vlx.ScfRestrictedDriver()
    scf_drv.ri_coulomb = True
    scf_drv.grid_level = 2
    scf_drv.xcfun = "pbe"
    scf_drv.dispersion = True
    scf_drv.conv_thresh = 1e-4
    scf_drv.ostream.mute()
    result = scf_drv.compute(mol, basis)
    results.append(result['scf_energy'])
    print(str("Energy: " + str(result['scf_energy'])) + " of structure " + str(i+1))
print("Finished")

Energy: -468.6172737471755 of structure 1
Energy: -614.7838668652547 of structure 2
Energy: -365.754794762156 of structure 3
Energy: -388.3555891917568 of structure 4
Energy: -349.72169546169556 of structure 5
Energy: -271.24245846278666 of structure 6
Energy: -251.3907906352136 of structure 7
Energy: -403.53856468432275 of structure 8
Energy: -330.4418302058259 of structure 9
Energy: -322.49895772910645 of structure 10
Finished
CPU times: user 2min 24s, sys: 1min 26s, total: 3min 50s
Wall time: 38 s


In [12]:
%%time
results = []
for i in range(10):
    mol = vlx.Molecule.read_smiles(smiles_list[i])
    rd_mol = Chem.MolFromSmiles(smiles_list[i])
    charge = Chem.GetFormalCharge(rd_mol)
    mol.set_charge(charge)
    basis = vlx.MolecularBasis.read(mol, 'def2-svpd')
    scf_drv = vlx.ScfRestrictedDriver()
    scf_drv.ri_coulomb = False
    scf_drv.grid_level = 2
    scf_drv.xcfun = "pbe"
    scf_drv.dispersion = True
    scf_drv.conv_thresh = 1e-4
    scf_drv.ostream.mute()
    result = scf_drv.compute(mol, basis)
    results.append(result['scf_energy'])
    print(str("Energy: " + str(result['scf_energy'])) + " of structure " + str(i+1))
print("Finished")

Energy: -468.61637204024396 of structure 1
Energy: -614.783630288532 of structure 2
Energy: -365.7546231019231 of structure 3
Energy: -388.35466981141553 of structure 4
Energy: -349.71992072122885 of structure 5
Energy: -271.24085174813456 of structure 6
Energy: -251.39091581015512 of structure 7
Energy: -403.5375939627715 of structure 8
Energy: -330.4332243670178 of structure 9
Energy: -322.49891845237386 of structure 10
Finished
CPU times: user 41min 48s, sys: 3min 6s, total: 44min 54s
Wall time: 3min 39s


In [13]:
%%time
results = []
for i in range(10):
    mol = vlx.Molecule.read_smiles(smiles_list[i])
    rd_mol = Chem.MolFromSmiles(smiles_list[i])
    charge = Chem.GetFormalCharge(rd_mol)
    mol.set_charge(charge)
    basis = vlx.MolecularBasis.read(mol, 'def2-tzvp')
    scf_drv = vlx.ScfRestrictedDriver()
    scf_drv.ri_coulomb = True
    scf_drv.grid_level = 2
    scf_drv.xcfun = "pbe"
    scf_drv.dispersion = True
    scf_drv.conv_thresh = 1e-4
    scf_drv.ostream.mute()
    result = scf_drv.compute(mol, basis)
    results.append(result['scf_energy'])
    print(str("Energy: " + str(result['scf_energy'])) + " of structure " + str(i+1))
print("Finished")

Energy: -469.0900576392116 of structure 1
Energy: -615.4089581369107 of structure 2
Energy: -366.12868547219745 of structure 3
Energy: -388.74695750526797 of structure 4
Energy: -350.0780876339713 of structure 5
Energy: -271.521580731587 of structure 6
Energy: -251.64669038618115 of structure 7
Energy: -403.9383164651258 of structure 8
Energy: -330.7598281402643 of structure 9
Energy: -322.8335255726159 of structure 10
Finished
CPU times: user 3min 32s, sys: 1min 46s, total: 5min 19s
Wall time: 50.2 s


In [14]:
%%time
results = []
for i in range(10):
    mol = vlx.Molecule.read_smiles(smiles_list[i])
    rd_mol = Chem.MolFromSmiles(smiles_list[i])
    charge = Chem.GetFormalCharge(rd_mol)
    mol.set_charge(charge)
    basis = vlx.MolecularBasis.read(mol, 'def2-tzvp')
    scf_drv = vlx.ScfRestrictedDriver()
    scf_drv.ri_coulomb = False
    scf_drv.grid_level = 2
    scf_drv.xcfun = "pbe"
    scf_drv.dispersion = True
    scf_drv.conv_thresh = 1e-4
    scf_drv.ostream.mute()
    result = scf_drv.compute(mol, basis)
    results.append(result['scf_energy'])
    print(str("Energy: " + str(result['scf_energy'])) + " of structure " + str(i+1))
print("Finished")

Energy: -469.08937903306406 of structure 1
Energy: -615.4105162886488 of structure 2
Energy: -366.12830531080846 of structure 3
Energy: -388.7469688053938 of structure 4
Energy: -350.0797667402313 of structure 5
Energy: -271.5212589971323 of structure 6
Energy: -251.6466081416732 of structure 7
Energy: -403.937030277997 of structure 8
Energy: -330.763758682828 of structure 9
Energy: -322.83345294995524 of structure 10
Finished
CPU times: user 2h 8min 39s, sys: 7min 55s, total: 2h 16min 34s
Wall time: 10min 53s
